In [1]:
import firebase_admin
from firebase_admin import credentials, db
import openmeteo_requests
import requests_cache
import pandas as pd
from retry_requests import retry
from datetime import datetime

# ================= CONFIGURASI =================
# 1. Lokasi & Waktu
LAT = -7.66  # Kebumen
LON = 109.65
# Format: YYYY-MM-DD
TARGET_DATE = "2025-11-29" 
START_TIME_WIB = "03:09:31"
END_TIME_WIB   = "07:46:31"
FORMAT_KEY = 'UNIX'  # <--- GANTI INI SESUAI DATABASE MU! (Saran: Coba UNIX dulu)

In [2]:
# 2. Firebase Config
# Ganti dengan path file JSON Anda
CRED_PATH = "/work/staklimjerukagung-firebase-admin.json" 
DB_URL = "https://staklimjerukagung-default-rtdb.asia-southeast1.firebasedatabase.app/"
# Path di database tempat data sensor disimpan
# Misal: sensors/kbm-05/logs
DB_NODE_PATH = "/auto_weather_stat/id-05/data" 

# ================= 1. SETUP KONEKSI =================
# Setup Firebase
if not firebase_admin._apps:
    cred = credentials.Certificate(CRED_PATH)
    firebase_admin.initialize_app(cred, {'databaseURL': DB_URL})

# Setup Open-Meteo
cache_session = requests_cache.CachedSession('.cache', expire_after=3600)
retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
client = openmeteo_requests.Client(session=retry_session)

In [ ]:
def get_data_interpolated():
    url = "https://archive-api.open-meteo.com/v1/archive" 
    
    # Menambahkan 'rain' dan 'surface_pressure'
    params = {
        "latitude": LAT, "longitude": LON,
        "start_date": TARGET_DATE, "end_date": TARGET_DATE,
        "hourly": [
            "temperature_2m",       # Index 0
            "relative_humidity_2m", # Index 1
            "dew_point_2m",         # Index 2
            "surface_pressure",     # Index 3
            "rain"                  # Index 4
        ],
        "timezone": "Asia/Jakarta"
    }
    
    print("🚀 Mengambil data Open-Meteo...")
    try:
        responses = client.weather_api(url, params=params)
        response = responses[0]
    except Exception as e:
        print(f"❌ Error API: {e}")
        return None

    # Proses Data
    hourly = response.Hourly()
    
    # HATI-HATI URUTAN INDEX HARUS SAMA DENGAN PARAMS DI ATAS
    df = pd.DataFrame({
        "date": pd.date_range(
            start=pd.to_datetime(hourly.Time(), unit="s", utc=True),
            end=pd.to_datetime(hourly.TimeEnd(), unit="s", utc=True),
            freq=pd.Timedelta(seconds=hourly.Interval()),
            inclusive="left"
        ),
        "temperature": hourly.Variables(0).ValuesAsNumpy(),
        "humidity": hourly.Variables(1).ValuesAsNumpy(),
        "dew": hourly.Variables(2).ValuesAsNumpy(),
        "pressure": hourly.Variables(3).ValuesAsNumpy(), # Tekanan Udara
        "rain": hourly.Variables(4).ValuesAsNumpy()      # Hujan
    })
    
    # 1. Konversi ke WIB
    df = df.set_index('date')
    df.index = df.index.tz_convert('Asia/Jakarta')
    
    # 2. Resample ke Menit & Interpolasi
    df = df.resample('1T').asfreq()
    
    # Interpolasi Linear (Halus untuk suhu/tekanan)
    df = df.interpolate(method='time')
    
    # 3. Potong Waktu
    start = f"{TARGET_DATE} {START_TIME_WIB}"
    end   = f"{TARGET_DATE} {END_TIME_WIB}"
    df = df.loc[start:end]
    
    # 4. Cleaning
    df['rain'] = df['rain'].fillna(0) 
    df = df.dropna()
    
    return df

def upload_fix(df):
    ref = db.reference(DB_NODE_PATH)
    print(f"🔥 Memulai Upload ({len(df)} data)...")
    
    for index, row in df.iterrows():
        # --- LOGIKA FORMAT KEY ---
        if FORMAT_KEY == 'UNIX':
            key = str(int(index.timestamp()))
        elif FORMAT_KEY == 'ISO':
            key = index.strftime('%Y-%m-%dT%H:%M:%S')
        else:
            key = index.strftime('%Y-%m-%d %H:%M:%S')
            
        # --- FORMAT TIMESTAMP (STRING MUTLAK) ---
        timestamp_str = index.strftime('%Y-%m-%d %H:%M:%S')

        # --- MENYIAPKAN DATA BODY ---
        data = {
            'temperature': round(float(row['temperature']), 2),
            'humidity': round(float(row['humidity']), 2),
            'dew': round(float(row['dew']), 2),
            'pressure': round(float(row['pressure']), 2), # Data Tekanan
            'rainrate': round(float(row['rain']), 2),
            'rainfall': round(float(row['rain']), 2),
            'timestamp': timestamp_str, # <--- String, bukan Object!
            # 'is_patched': True # Opsional, aktifkan jika perlu penanda
        }
        
        try:
            ref.child(key).update(data)
            # Print log per 10 menit data biar ga spam
            if int(index.second) == 0 and int(index.minute) % 10 == 0:
                print(f"   ✅ Progress: {timestamp_str} -> Key: {key}")
        except Exception as e:
            print(f"❌ Gagal pada {key}: {e}")
            
    print("✨ Selesai Upload!")

# EKSEKUSI UTAMA
if __name__ == "__main__":
    df_hasil = get_data_interpolated()
    
    if df_hasil is not None and not df_hasil.empty:
        print("\n" + "="*40)
        print("👀 PREVIEW DATA YANG AKAN DI-UPLOAD")
        print("="*40)
        
        # Tampilkan 5 baris pertama dengan kolom yang relevan
        preview_df = df_hasil.copy()
        preview_df['rainrate'] = preview_df['rain'].round(2)
        preview_df['timestamp_str'] = preview_df.index.strftime('%H:%M:%S')
        
        print(preview_df[['temperature', 'humidity', 'pressure', 'rainrate']].head())
        print("..." + str(len(df_hasil) - 10) + " data lainnya...")
        print(preview_df[['temperature', 'humidity', 'pressure', 'rainrate']].tail())
        
        print("\n⚠️ PERHATIAN:")
        print(f"1. Total Data: {len(df_hasil)} baris")
        print(f"2. Database  : {DB_NODE_PATH}")
        print(f"3. Format Key: {FORMAT_KEY}")
        
        x = input("\nApakah data di atas sudah benar dan siap upload? (y/n): ")
        if x.lower() == 'y':
            upload_fix(df_hasil)
        else:
            print("Dibatalkan.")
    else:
        print("Data kosong atau gagal diambil dari Open-Meteo.")

🚀 Mengambil data Open-Meteo...

👀 PREVIEW DATA YANG AKAN DI-UPLOAD
                           temperature   humidity     pressure  rainrate
date                                                                    
2025-11-29 03:10:00+07:00    22.502001  87.933418  1006.223877       0.0
2025-11-29 03:11:00+07:00    22.494501  87.932777  1006.230408       0.0
2025-11-29 03:12:00+07:00    22.487001  87.932137  1006.236938       0.0
2025-11-29 03:13:00+07:00    22.479502  87.931488  1006.243469       0.0
2025-11-29 03:14:00+07:00    22.472002  87.930847  1006.250061       0.0
...267 data lainnya...
                           temperature   humidity     pressure  rainrate
date                                                                    
2025-11-29 07:42:00+07:00    26.707001  70.081123  1008.691895       0.0
2025-11-29 07:43:00+07:00    26.738667  69.935982  1008.703918       0.0
2025-11-29 07:44:00+07:00    26.770334  69.790840  1008.715942       0.0
2025-11-29 07:45:00+07:00    26.80

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=200afad5-8991-4e20-8a55-8751c7aff3b5' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>